In [1]:
import os
from os.path import basename, dirname
import sys
import string
import glob

import numpy as np

from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu
from nltk import word_tokenize

from scipy import spatial
import gensim
from gensim.models import Word2Vec

In [2]:
word2vec_path = "/share03/song/word2vec/en/model.txt"
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, unicode_errors='ignore')                                                                 

print ("word2vec model loaded")

word2vec model loaded


In [3]:
en_folder =  "./03_length_clean"
trans_en_folder = "./05_separate_docs"                                                
output_folder = "./06_ids"

if (os.path.exists(en_folder) == 0):
    print ("No en folder!")
    exit(0)
    
if (os.path.exists(trans_en_folder) == 0):
    print ("No trans_en folder!")
    exit(0)

if (os.path.exists(output_folder) == 0):
    os.makedirs(output_folder)
    
names = glob.glob("{}/*.en.txt".format(en_folder))
new_names = [''.join(basename(name).strip().split('.')[:-2]) for name in names]

new_names.sort()
print (len(new_names))

818


In [4]:
def get_cos(sentence):
    res = np.zeros(100)
    num_of_words = len(sentence)
    for word in sentence:
        try:
            res += model[word]
        except:
            num_of_words -= 1
    res/=num_of_words
    return res                                                                  

def get_cos_similarity(hy, ref):
    hy_cos = get_cos(hy)
    ref_cos = get_cos(ref)
    similarity = 1 - spatial.distance.cosine(hy_cos, ref_cos)
    return similarity

def check_position_similarity(i, trans_len, j, en_len):
    relative_trans_loc = float(i)/trans_len
    relative_en_loc = float(j)/en_len
    relative_err = abs(relative_trans_loc - relative_en_loc)
    
    return relative_err
    
    if (relative_err < 0.3):
        return True
    else:
        return False

In [5]:
dp = []
flag = []
decide = [] # 1: match i with j, 2:j go next 3:i go next
match = []
cos_mat = []
trans_len = 0
en_len = 0
len_trans_en = []
len_en = []
en_tokenized_lines = []
trans_en_tokenized_lines = []

def length_check(leni, lenj):
    if (float(leni)/lenj < 0.5 or float(lenj)/leni < 0.5):
        return False
    else:
        return True

def get_words_num(sentence):
    return (len(sentence))
    
def dp(i ,j):
    
    if (i>=trans_len or j>=en_len):
        return 0
    if (flag[i][j] == 1):
        return f[i][j]
    flag[i][j] = 1

    #select_i_j = dp(i+1, j+1) + cos_mat[i][j] - check_position_similarity(i, trans_len, j, en_len)/10
    if (cos_mat[i][j] > 0.92):
        if (length_check(len_trans_en[i], len_en[j])):
            select_i_j = dp(i+1, j+1) + cos_mat[i][j]# - check_position_similarity(i, trans_len, j, en_len)/10
        else:
            select_i_j = -1
            #print (cos_mat[i][j])
            #print (trans_en_tokenized_lines[i])
            #print (en_tokenized_lines[j])
    else:
        select_i_j = -1
        
    select_i_afterj = dp(i, j+1)
    select_afteri_j = dp(i+1, j)
    if (select_i_j > select_i_afterj and select_i_j > select_afteri_j):
        f[i][j] = select_i_j
        decide[i][j] = 1
    elif (select_i_afterj > select_afteri_j):
        f[i][j] = select_i_afterj
        decide[i][j] = 2
    else:
        f[i][j] = select_afteri_j
        decide[i][j] = 3
    
    return f[i][j]
    
def align(en_lines, trans_en_lines, ids_file):
    # initialize dp array
    global f, flag, decide, cos_mat, trans_len, en_len, match, len_en
    global len_trans_en, en_tokenized_lines, trans_en_tokenized_lines
    trans_len = len(trans_en_lines)
    en_len = len(en_lines)
    f = [[0 for i in range(en_len)] for j in range(trans_len)]
    flag = [[0 for i in range(en_len)] for j in range(trans_len)]
    decide = [[0 for i in range(en_len)] for j in range(trans_len)]
    cos_mat = [[0 for i in range(en_len)] for j in range(trans_len)]
    match = [-1 for j in range(trans_len)]
    len_trans_en = []
    len_en = []
    
    en_tokenized_lines = []
    for line in en_lines:
        new_line = ''
        for c in line:
            if (ord(c)<128):
                new_line += string.lower(c)
        new_line = word_tokenize(new_line)
        len_en.append(get_words_num(new_line))
        en_tokenized_lines.append(new_line)

    trans_en_tokenized_lines = []
    for line in trans_en_lines:
        new_line = ''
        for c in line:
            if (ord(c)<128):
                new_line += string.lower(c)
        new_line = word_tokenize(new_line)
        len_trans_en.append(get_words_num(new_line))
        trans_en_tokenized_lines.append(new_line)
    
    #print (trans_en_tokenized_lines)
    #print (len_trans_en)
    #print (en_tokenized_lines)
    #print (len_en)
        

    en_len = len(en_tokenized_lines)
    trans_len = len(trans_en_tokenized_lines)

    

    for i in range(trans_len):                                                                                                                                                                              
        hy = trans_en_tokenized_lines[i]
        max_cos = 0
        idx = -1
        for j in range(en_len):
            ref = en_tokenized_lines[j]
            cos_mat[i][j] = get_cos_similarity(hy, ref)
    
    
    dp(0, 0)
    i = 0
    j = 0
    while (i<trans_len and j<en_len):
        if (decide[i][j] == 1):
            match[i] = j
            i+=1
            j+=1
        elif (decide[i][j] == 2):
            j+=1
        else:
            match[i] = -1
            i+=1
            
    for i in range(trans_len):
        #print (i, match[i])
        if (match[i] >= 0):
            #print (trans_en_lines[i])
            #print (en_lines[match[i]])
            output = "{} {} {}\n".format(i,match[i],cos_mat[i][match[i]])
            #print (output)
            
            ids_file.write(output)

In [6]:
for i, name in enumerate(new_names):
    en_name = os.path.join(en_folder, name + '.en.txt')
    trans_en_name = os.path.join(trans_en_folder, name + '.trans_en.txt')
    ids_name = os.path.join(output_folder, name + '.ids.txt')

    print (i, name)
    with open(en_name, "r") as f:
        en_lines = f.readlines()
    with open(trans_en_name, "r") as f:
        trans_en_lines = f.readlines()
    with open(ids_name, "w") as f:
        align(en_lines, trans_en_lines, f)

(0, '1062_01_welcome-to-the-course')
(1, '1063_02_what-to-expect')
(2, '1064_01_the-3-traps-of-communication')
(3, '1066_04_second-trap-speaking-in-formal-language')
(4, '1067_06_third-trap-speaking-with-too-many-details')
(5, '1068_01_misconceptions-pros-cons')
(6, '1069_01_three-types-of-remote-teams')
(7, '1076_02_picture-superiority-effect')
(8, '1080_04_uninformed-empathy-pov')
(9, '1082_02_achievement')
(10, '1085_01_questions')
(11, '1089_01_positional-vs-principled-bargaining')
(12, '1090_01_why')
(13, '1091_02_how')
(14, '1096_05_veggies')
(15, '1100_05_meaningful-numbers')
(16, '1121_03_motivation')
(17, '1126_01_introduction')
(18, '1132_02_adjusting-model-assumptions')
(19, '1136_02_adjusting-model-assumptions')
(20, '1143_08_clase-magistral-ritmo-melodico-y-ritmos-de-letra-parte-4')
(21, '1147_01_1-1-course-introduction')
(22, '115_01_solving-the-sum-of-two-digits-programming-challenge-screencast')
(23, '116_02_coming-up')
(24, '117_01_largest-number')
(25, '119_08_merge-s

(182, '4107_03_how-differently-do-different-balls-fall')
(183, '4108_04_how-would-a-ball-fall-on-the-moon')
(184, '4109_01_how-does-a-falling-ball-move-after-it-is-dropped')
(185, '4110_02_how-can-a-ball-move-upward-and-still-be-falling')
(186, '4111_03_how-does-a-balls-horizontal-motion-affect-its-fall')
(187, '4112_01_falling-balls-summary')
(188, '4113_01_ramps-introduction')
(189, '4114_02_why-doesn-t-a-wagon-fall-through-a-sidewalk')
(190, '4115_03_why-does-a-sidewalk-perfectly-support-a-wagon')
(191, '4116_01_how-does-a-wagon-move-as-you-let-it-roll-freely-on-a-ramp')
(192, '4117_02_why-is-it-more-exhausting-to-lift-a-wagon-up-than-to-lower-a-wagon-down')
(193, '4118_03_why-is-it-easier-to-pull-a-wagon-up-a-ramp-than-to-lift-it-up-a-ladder')
(194, '4119_01_ramps-summary')
(195, '4120_01_seesaws-introduction')
(196, '4121_02_how-does-a-balanced-seesaw-move')
(197, '4122_03_why-does-a-seesaw-need-a-pivot')
(198, '4123_04_why-does-a-lone-seesaw-rider-plummet-to-the-ground')
(199, '4

(358, '5623_01_learning-with-large-datasets')
(359, '5624_02_stochastic-gradient-descent')
(360, '5625_03_mini-batch-gradient-descent')
(361, '5626_04_stochastic-gradient-descent-convergence')
(362, '5628_02_map-reduce-and-data-parallelism')
(363, '5629_01_problem-description-and-pipeline')
(364, '5630_02_sliding-windows')
(365, '5631_03_getting-lots-of-data-and-artificial-data')
(366, '5632_04_ceiling-analysis-what-part-of-the-pipeline-to-work-on-next')
(367, '56_01_jian-jie-introduction')
(368, '5725_01_lecture-0-welcome')
(369, '57_01_jian-jie-introduction')
(370, '5814_03_batchelor-commercialism-and-mindfulness')
(371, '5817_07_1-7-should-you-listen-to-music-when-you-re-studying')
(372, '5826_03_welcome-to-this-course-and-specialization')
(373, '5827_03_what-you-ll-be-able-to-do')
(374, '5828_06_inspecting-the-model-coefficients-learned')
(375, '5829_02_analyzing-the-sentiment-of-reviews-a-case-study-in-classification')
(376, '5832_02_document-retrieval-a-case-study-in-clustering-a

(535, '6599_01_tuo-yan-zheng-jian-jie')
(536, '6601_01_zu-kuai-gai-lun')
(537, '6602_01_tuo-yan-zheng-yu-ji-yi-gai-lun')
(538, '696_03_video-1-course-overview')
(539, '697_01_video-1-introduction')
(540, '7045_03_introduction')
(541, '7105_09_master-class-melodic-rhythm-and-lyric-rhythm-part-4')
(542, '7123_01_small-talk-conversational-vocabulary')
(543, '7124_01_key-pronunciation-stress-intonation')
(544, '7125_01_the-elevator-speech')
(545, '7126_01_basics-of-meeting-online')
(546, '7127_01_group-discussion-language')
(547, '7129_01_lets-learn-telephone-language')
(548, '7131_01_phone-role-play')
(549, '7132_01_first-impressions-body-language')
(550, '7133_01_key-pronunciation-ed-s-endings')
(551, '7134_01_interview-role-play')
(552, '7135_01_organizing-your-pitch-3-tells')
(553, '7136_01_more-about-stress-intonation')
(554, '7168_01_lecture-1-introduction-to-monte-carlo-algorithms')
(555, '7170_01_lecture-2-hard-disks-from-classical-mechanics-to-statistical-mechanics')
(556, '7171_0

/share03/song/MT/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':


(558, '7173_01_lecture-5-density-matrices-and-path-integrals')
(559, '7175_01_lecture-7-quantum-indiscernability-and-bose-einstein-condensation')
(560, '7176_01_lecture-8-ising-model-from-enumeration-to-cluster-monte-carlo-simulations')
(561, '7180_01_lecture-10-the-alpha-and-the-omega-of-monte-carlo')
(562, '7213_01_the-realities-of-admission-understanding-options')
(563, '7258_02_economic-growth-and-progress')
(564, '7260_01_economic-development-is-new')
(565, '7261_01_the-great-waves-of-technological-change')
(566, '7292_03_how-can-i-best-support-younger-students-to-use-classroom-social-media')
(567, '7351_01_1-1-time-value-of-money-tvm')
(568, '7352_02_1-2-simple-future-value-fv')
(569, '7353_03_1-3-simple-fv')
(570, '7357_01_1-1-time-value-of-money-tvm')
(571, '7358_02_1-2-simple-future-value-fv')
(572, '7359_03_1-3-simple-fv')
(573, '7376_01_1-1-el-trabajo-productivo-y-el-improductivo')
(574, '7378_01_2-1-el-mito-del-equilibrio-entre-trabajo-y-vida-personal')
(575, '7412_01_why-a

(715, '7694_08_more-reflections-on-customer-centricity')
(716, '7695_09_questions-on-customer-centricity')
(717, '7696_01_introduction-and-execution')
(718, '7697_02_online-offline-competition')
(719, '7698_03_friction')
(720, '7700_05_the-long-tail-part-2')
(721, '7701_06_preference-isolation')
(722, '7702_07_how-internet-retailing-startups-grow')
(723, '7703_08_customers-and-digital-marketing')
(724, '7704_09_influence-and-how-information-spreads')
(725, '7705_10_pricing-strategies-1-introduction')
(726, '7706_11_pricing-strategies-2-customer-factors')
(727, '7707_12_the-7ms')
(728, '7708_01_brand-messaging-communication')
(729, '7709_02_brand-elements-choosing-a-brand-name')
(730, '7710_03_brand-elements-color-taglines')
(731, '7711_04_brand-elements-packaging')
(732, '7712_05_brand-elements-persuasion')
(733, '7713_06_repositioning-a-brand')
(734, '7714_01_the-brand-named-jennifer')
(735, '7715_02_awesomeness-tv')
(736, '7716_03_playboy-in-china')
(737, '7717_04_crocs')
(738, '7718